In [ ]:
from beir.datasets.data_loader import GenericDataLoader

In [3]:
ClapNQcorpus, ClapNQqueries, ClapNQqrels = GenericDataLoader("dataset/clapnq").load(split="train")

In [5]:
import json, pathlib

beir_corpus = pathlib.Path("dataset/clapnq/corpus.jsonl")
jsonl_dir   = pathlib.Path("indexes/clapnq-jsonl")
jsonl_dir.mkdir(parents=True, exist_ok=True)
out = jsonl_dir/"docs.jsonl"

with open(beir_corpus, "r", encoding="utf-8") as fin, open(out, "w", encoding="utf-8") as fout:
    for line in fin:
        d = json.loads(line)
        title = (d.get("title") or "").strip()
        text  = (d.get("text") or "").strip()
        
        if title and text.startswith(title):
            contents = text
        else:
            contents = (title + "\n" + text).strip()

        fout.write(json.dumps(
            {"id": d["_id"], "contents": contents},
            ensure_ascii=False
        ) + "\n")

print("Wrote:", out)

Wrote: indexes/clapnq-jsonl/docs.jsonl


In [7]:
from pyserini.search.lucene import LuceneSearcher
from beir.retrieval.evaluation import EvaluateRetrieval

searcher = LuceneSearcher("indexes/clapnq-lucene")
searcher.set_bm25(k1=0.9, b=0.4)

results = {}
for qid, qtext in ClapNQqueries.items():
    hits = searcher.search(qtext, k=10)
    results[qid] = {h.docid: float(h.score) for h in hits}

evaluator = EvaluateRetrieval(None)
ndcg, _map, recall, precision = evaluator.evaluate(ClapNQqrels, results, k_values=[1,3,5,10])
print("nDCG:", ndcg)
print("recall:", recall)

ImportError: cannot import name '_lazywhere' from 'scipy._lib._util' (/Users/leeediazk/miniconda3/envs/mtrag/lib/python3.11/site-packages/scipy/_lib/_util.py)